In [1]:
import torch 
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms

In [3]:
training_data=datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [4]:
test_data=datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [15]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(training_data,32,True, drop_last=True)
test_dataloader=DataLoader(test_data,32,False, drop_last=True)

In [6]:
imgs, labels=next(iter(train_dataloader))
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
imgs.shape

torch.Size([32, 3, 32, 32])

In [7]:
from torch import nn


In [8]:
class CIFAR10(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*64 , 
                      out_features=output_shape)
        )
    
    def forward(self, x):
   
        x = self.layer1(x)

        x = self.layer2(x)
       
        x = self.layer3(x)
        
        return x

model=CIFAR10(input_shape=3,hidden_units=10,output_shape=len(classes))

In [9]:
dummy=torch.randn([1,3,32,32])
model(dummy)

tensor([[-0.0293,  0.0322, -0.0461,  0.0474, -0.0055,  0.0732, -0.0368,  0.1202,
          0.1034, -0.0642]], grad_fn=<AddmmBackward0>)

In [10]:
from tqdm import tqdm
import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)


helper_functions.py already exists, skipping download


In [19]:
from helper_functions import accuracy_fn
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.1)

In [20]:
epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    train_loss = 0
    
    # Training loop
    for batch, (X, y) in enumerate(train_dataloader):
        model.train()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")
    
    train_loss = train_loss / len(train_dataloader)  # Non-in-place division
    
    # Evaluation loop
    test_loss, test_acc = 0, 0
    model.eval()
    
    with torch.inference_mode():
        for X, y in test_dataloader:
            test_pred = model(X)
            test_loss += loss_fn(test_pred, y).item()  # Use .item() to get a Python number
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))  # Use .item() here as well
    
    test_loss = test_loss / len(test_dataloader)  # Non-in-place division
    test_acc = test_acc / len(test_dataloader)    # Non-in-place division
    
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Epoch: 0
-------
Looked at 0/50000 samples
Looked at 12800/50000 samples
Looked at 25600/50000 samples
Looked at 38400/50000 samples


 33%|████████████████████████████                                                        | 1/3 [00:37<01:14, 37.05s/it]


Train loss: 2.30377 | Test loss: 2.30292, Test acc: 9.99%

Epoch: 1
-------
Looked at 0/50000 samples
Looked at 12800/50000 samples
Looked at 25600/50000 samples
Looked at 38400/50000 samples


 67%|████████████████████████████████████████████████████████                            | 2/3 [01:14<00:37, 37.57s/it]


Train loss: 2.30345 | Test loss: 2.30324, Test acc: 10.01%

Epoch: 2
-------
Looked at 0/50000 samples
Looked at 12800/50000 samples
Looked at 25600/50000 samples
Looked at 38400/50000 samples


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:53<00:00, 37.81s/it]


Train loss: 2.30336 | Test loss: 2.30305, Test acc: 10.01%

